In [1]:
import struct
from math import log, exp
from array import array
import numpy as np

In [2]:
def read(fname_img, fname_lbl, n):
    f = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", f.read(8))
    lbl = array("b", f.read())
    f.close()

    f = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", f.read(16))
    img = array("B", f.read())
    f.close()

    tmp = [1]
    img_matrix = []
    for x in img.tolist():
        tmp.append(x)
        if len(tmp) == 785:
            img_matrix.append(tmp)
            tmp = [1]
            if (len(img_matrix) == n):
                break

    return lbl.tolist()[:n], img_matrix

In [3]:
train_size = 60000
test_size = 20000

train_lbl, train_img = read("train-images-idx3-ubyte", "train-labels-idx1-ubyte", train_size)
test_lbl, test_img = read("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte", test_size)

In [4]:
print len(train_lbl), len(train_img)
print len(test_lbl), len(test_img)

60000 60000
10000 10000


In [5]:
def cal_accuracy(result, lbl):
    correct = sum([int(result[i] == lbl[i]) for i in range(len(result))])
    return float(correct)/len(result)

In [ ]:
from sklearn import neighbors
from sklearn.metrics import confusion_matrix

clf = neighbors.KNeighborsClassifier(n_neighbors=1, weights='distance', p=2) #euclidean_distance
clf.fit(train_img, train_lbl)
result = clf.predict(test_img)
print "Accuracy: ", cal_accuracy(result, test_lbl)
print confusion_matrix(result, test_lbl)
print clf

Accuracy:  0.9691
[[ 973    0    7    0    0    1    4    0    6    2]
 [   1 1129    6    1    7    1    2   14    1    5]
 [   1    3  992    2    0    0    0    6    3    1]
 [   0    0    5  970    0   12    0    2   14    6]
 [   0    1    1    1  944    2    3    4    5   10]
 [   1    1    0   19    0  860    5    0   13    5]
 [   3    1    2    0    3    5  944    0    3    1]
 [   1    0   16    7    5    1    0  992    4   11]
 [   0    0    3    7    1    6    0    0  920    1]
 [   0    0    0    3   22    4    0   10    5  967]]
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=1, p=2, weights='distance')


In [14]:
from sklearn import svm
from sklearn.metrics import confusion_matrix

clf = svm.LinearSVC()
clf.fit(train_img, train_lbl)
result = clf.predict(test_img)
print "Accuracy: ", cal_accuracy(result, test_lbl)
print confusion_matrix(result, test_lbl)
print clf

Accuracy:  0.8864
[[ 940    0   10   14    4   15   20    2   10    9]
 [   0 1092   11    5    0    4    4    0   17    2]
 [   3    5  881   27    4    4    9   31    8    3]
 [   2    0   28  867    3   48    1    4   35   15]
 [   1    1    8    1  890   10    9   11   19   51]
 [   2    0    5   33    1  732   11    4   69    6]
 [   4    3   16    5   14   23  891    1   11    0]
 [   2    2    9   15    1    7    0  915   10   29]
 [  24   28   60   24    9   35   11    8  774   12]
 [   2    4    4   19   56   14    2   52   21  882]]
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.initialize_all_variables())

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

print 
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
result = open('result.txt', 'wb')
for i in range(20000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    if i % 100 == 0:
        train_loss = cross_entropy.eval(feed_dict={x: mnist.train.images, y_: mnist.train.labels, keep_prob: 1.0})
        test_loss = cross_entropy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
        result.write(str(i + 1) + ' ' + str(train_loss) + ' ' + str(test_loss) + '\n')
        print i, train_loss, test_loss

result.close()
test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
print test_accuracy

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

